In [2]:
import numpy as np
import pandas as pd
import scipy as sci
import matplotlib.pyplot as plt
import os
import altair as alt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from sklearn import set_config
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, cross_validate, train_test_split


# alt.data_transformers.enable('vegafusion')
# alt.renderers.enable('default')
# set_config(transform_output="pandas")

print("packages imported")

packages imported


In [3]:
url_players='https://drive.google.com/file/d/1Mw9vW0hjTJwRWx0bDXiSpYsO3gKogaPz/edit'
url_players='https://drive.google.com/uc?id=' + url_players.split('/')[-2]
players = pd.read_csv(url_players)

url_sessions='https://drive.google.com/file/d/14O91N5OlVkvdGxXNJUj5jIsV5RexhzbB/edit'
url_sessions='https://drive.google.com/uc?id=' + url_sessions.split('/')[-2]
sessions = pd.read_csv(url_sessions)

experience_mapping = {
    'Beginner': 1,
    'Amateur': 2,
    'Regular': 3,
    'Veteran': 4,
    'Pro': 5
}

sessions['start_time'] = pd.to_datetime(sessions['start_time'], format="%d/%m/%Y %H:%M")
sessions['end_time'] = pd.to_datetime(sessions['end_time'], format="%d/%m/%Y %H:%M")

sessions['session_length'] = (sessions['end_time'] - sessions['start_time']).dt.total_seconds() / 3600
sessions['original_session_length'] = (sessions['original_end_time'] - sessions['original_start_time'])

player_sessions = sessions.groupby('hashedEmail').agg(
    number_sessions=('session_length', 'size'), 
    mean_session_length=('session_length', 'mean'), 
    sd_session_length=('session_length', 'std') 
).reset_index()

players_combined = pd.merge(players, player_sessions, on='hashedEmail', how='left')
players_combined['experience_val'] = players_combined['experience'].map(experience_mapping)
players_combined['subscribe_binary'] = players_combined['subscribe'].astype(int)
players_combined = players_combined.dropna(subset=['subscribe_binary', 'experience_val', 'age', 'number_sessions', 'played_hours'])
players_combined

,experience,subscribe,hashedEmail,played_hours,name,gender,age,individualId,organizationName,number_sessions,mean_session_length,sd_session_length,experience_val,subscribe_binary
0,Pro,True,f6daba428a5e19a3d47574858c13550499be23603422e6...,30.3,Morgan,Male,9,NaN,NaN,27.0,1.246296,0.902162,5,1
1,Veteran,True,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa9397...,3.8,Christian,Male,17,NaN,NaN,3.0,1.416667,1.233671,4,1
2,Veteran,False,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3...,0.0,Blake,Male,17,NaN,NaN,1.0,0.083333,NaN,4,0
3,Amateur,True,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4f...,0.7,Flora,Female,21,NaN,NaN,1.0,0.833333,NaN,2,1
4,Regular,True,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb...,0.1,Kylie,Male,21,NaN,NaN,1.0,0.150000,NaN,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,Veteran,True,ba24bebe588a34ac546f8559850c65bc90cd9d51b82158...,0.1,Gabriela,Female,44,NaN,NaN,1.0,0.183333,NaN,4,1
192,Veteran,False,71453e425f07d10da4fa2b349c83e73ccdf0fb3312f778...,0.3,Pascal,Male,22,NaN,NaN,1.0,0.350000,NaN,4,0
193,Amateur,False,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db29...,0.0,Dylan,Prefer not to say,17,NaN,NaN,1.0,0.083333,NaN,2,0
194,Amateur,False,f19e136ddde68f365afc860c725ccff54307dedd13968e...,2.3,Harlow,Male,17,NaN,NaN,6.0,0.497222,0.733592,2,0


In [4]:
players_training, players_testing = train_test_split(
   players_combined, test_size=0.3, random_state=2000 
)

X_train = players_training[["age"]]
y_train = players_training['played_hours']

X_test = players_testing[["age"]] 
y_test = players_testing['played_hours'] 

players_pipe = make_pipeline(
    StandardScaler(),
    KNeighborsRegressor()
)

marathon_cv = pd.DataFrame(
    cross_validate(
        players_pipe,
        X_train,
        y_train,
        scoring="neg_root_mean_squared_error",
        return_train_score=True
    )
)

param_grid = {'kneighborsregressor__n_neighbors': range(1, 30)}

player_tuned = GridSearchCV(
    players_pipe,
    param_grid,
    scoring="neg_root_mean_squared_error",
    cv=5,
    n_jobs=-1
)

player_results = pd.DataFrame(
    player_tuned.fit(X_train, y_train).cv_results_
)

player_min = player_tuned.best_params_
player_best_RMSPE = -player_tuned.best_score_

player_min

{'kneighborsregressor__n_neighbors': 23}

In [5]:
k = 23
knn_model = KNeighborsRegressor(n_neighbors=k)
knn_model.fit(X_train, y_train)

y_pred = knn_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared:", r2)

Mean Squared Error (MSE): 835.6586384439357
Root Mean Squared Error (RMSE): 28.907760868734467
R-squared: 0.058996348109105146


In [ ]:
plot_data = pd.DataFrame({
    'Age': X_test.squeeze(),
    'Actual Played Hours': y_test,
    'Predicted Played Hours': y_pred
})

scatter = alt.Chart(plot_data).mark_circle(size=60).encode(
    x=alt.X('Age', title='Age'),
    y=alt.Y('Actual Played Hours', title='Played Hours'),
    tooltip=['Age', 'Actual Played Hours', 'Predicted Played Hours']
).properties(
    title=f'k-NN Regression: Age vs Played Hours (K={k})'
)

line = alt.Chart(plot_data).mark_line(color='red').encode(
    x='Age',
    y='Predicted Played Hours'
)

final_plot = scatter + line

final_plot.show()

alt.LayerChart(...)